In [43]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [44]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [45]:
with open('wiz_of_oz.txt','r', encoding = 'utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [46]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode("Hello")

print(encoded_hello)

decoded_hello = decode(encoded_hello)
print(decoded_hello)

[32, 58, 65, 65, 68]
Hello


In [47]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:200])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50,  9,  1, 39, 50, 37, 25,  1, 39,
        30,  1, 39, 50,  9,  1, 29, 44, 27, 11,  0,  0,  1,  1, 33, 36, 36, 45,
        43, 44, 42, 25, 44, 29, 28,  1, 26, 49,  1, 34, 39, 32, 38,  1, 42, 11,
         1, 38, 29, 33, 36, 36,  0,  0,  1,  1, 26, 39, 39, 35, 43,  1, 39, 30,
         1, 47, 39, 38, 28, 29, 42,  1, 47, 33, 36, 36, 33, 25, 37,  1, 37, 39,
        42, 42, 39, 47,  1,  4,  1, 27, 39, 11,  9,  1, 33, 38, 27, 11,  1, 38,
        29, 47])


In [49]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return(x, y)
x,y = get_batch('train')
print('inputs:')
print(x)
print('outputs:')
print(y)
    

tensor([35328, 47996, 80966,  6223])
inputs:
tensor([[68, 59,  1, 73, 61, 58,  1, 55],
        [73, 58, 71,  1, 59, 71, 68, 66],
        [57,  1, 59, 68, 65, 65, 68, 76],
        [72, 69, 54, 69, 58, 71, 72,  1]], device='cuda:0')
outputs:
tensor([[59,  1, 73, 61, 58,  1, 55, 54],
        [58, 71,  1, 59, 71, 68, 66,  1],
        [ 1, 59, 68, 65, 65, 68, 76, 58],
        [69, 54, 69, 58, 71, 72,  1, 62]], device='cuda:0')


In [50]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [51]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


GZnFtH
x76W?k(wn2R. ?gi2&_;﻿cG_yv8vR&.Izo1*7OySPRq1))L&4gbIs]4*t8pE8rto:rn14UDRk"gQm!bo00vlPUlip)&xicvK
w0Tok
f_2vUVQPZ7hhVy9s]&x4Y4:ts﻿:FBEop7qkxGlRDdpeghwTjto:'ino:RZN _8KVNF3L(XOcOj﻿cDvZ);_Fty:-,Tbo:WC__K)Qy(?pIn UKV7td'cmmBbLN[BN*E1m7-﻿uvA,'C!oZEbHecIIN[yhwTJXeJSd(4vqL8hfmOZoHJ(4UlVn? nl"dW
﻿*RtJ2oW?"1*Qe jBlnbdXKZ[H[)?kic3h3sgatj0Nv?gbLlvme-js﻿x,XcjX5H﻿uRZ-TArwInaT*Jr_s﻿:  mI0CeKzZ0:T 4UZpAr'46Mc*[(U;00"lMhr(eU;ET6nRF 5OZno;v-C9K4uNhD_OoB"PfIVNwvYrKvlLY[9C*]lqyYK2(UMP"
0 O?
"cIl&Y]v&x1*sc:I


In [54]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([159992, 141584,  50120, 122392])
tensor([ 23716,  69163, 153399,  33887])
tensor([ 31407, 178584,  26998,  68908])
tensor([127408, 164958,  59899,  58820])
tensor([  8579, 183882, 109677,  26662])
tensor([83313, 54588, 58761, 73503])
tensor([ 96662, 104713,   1912, 107127])
tensor([179369, 146131,  79324, 170772])
tensor([ 81289, 153587, 113848,  72006])
tensor([ 11127, 175571, 103565,  64253])
tensor([144952,  36297, 151708, 182349])
tensor([ 68696,  16208, 141042, 166774])
tensor([ 9811, 64580, 58463, 25881])
tensor([ 76982,  23619, 129421,  25914])
tensor([ 53172,  43159, 101357, 167755])
tensor([137377, 108249, 107739, 129620])
tensor([185791,  49499,  99058,  14773])
tensor([180744,  77954,  75261, 165005])
tensor([ 98885, 108189, 132087,  77322])
tensor([ 28386, 172248,  89053, 128174])
tensor([ 46964,  43846, 115656,   1325])
tensor([103208,  86716,  17742,  85376])
tensor([ 66368, 147532,  81417, 117488])
tensor([ 33718, 129993,  28672,  42589])
tensor([133318,  49238, 

In [28]:
context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())  
print(generated_chars)


tthlvoXL'!MU]iz10of
C2uwOZQiz*xS.(H._Ne-X?2"We;X6r;yZxv[!Z0]D0, meve, 4Z0wema"O
"
Tqulen anLI.67Jsl; 
" J4I__NNLv!)k5f f﻿xrXy?gN;3XD01?2Y0qX2Udetlp!rermor gs aca&﻿*_NVQ6QA02﻿xc4TUh]bA2Xps bXdewhlMEeainl-8[Xbu:lu0s cath._aiv.TH&y[V i&zM, klDCjZ64'P-V;PVYCoMbpa[t2ve.]ezLadL'ttGero3eme,"ikfa[uUwoutH&.-"
EyhL﻿_HOToD0NV]he,FrtCic4ZathirinveS, r f!"I(gotoQ,"WUZ)LI9EaNgs?:jUampg!Y1Q5limul grc"X**2t
JeKZUgotVHGhe'Q(-K,iy,Lad0
bce
d.VB﻿Lato, fjBil
)ea.F:Efupo b il dCOGTheq:OvewirqVNTXk﻿gi._1lr wLq, il CG
